### Tela inicial
* Campos para as credenciais
* Teste de conexão
* Log no cache das credenciais


### Menu de SELECT
* Lista com todos os bancos para selecionar
* Campo para a query e botão de submit
* Retorna grid com o resultado da query + db_name


### Menu de DDL
* Lista com todos os bancos para selecionar
* Campo para a query e botão de submit
* Retorna os resultados escritos nome do banco + output

In [6]:
import pyodbc
from pyodbc import ProgrammingError
import time 

In [11]:
class Conexao:

    def __init__(self, server, database = 'master', user = None, password = None):
        self.server = server
        self.database = database
        self.user = user
        self.password = password
        
    def test_azure_connection(self):
        if self.user is not None and self.password is not None:
            conn_str = (
            "DRIVER={ODBC Driver 17 for SQL Server};"
            f"SERVER={server};DATABASE={self.database};UID={self.user};PWD={self.password};"
            )
            
        elif not self.user and not self.password:
            conn_str = 'DRIVER={SQL Server Native Client 11.0};SERVER='+self.server+';DATABASE='+self.database+';Trusted_Connection=yes;'

        try:
            with pyodbc.connect(conn_str) as conn:
                cursor = conn.cursor()
                cursor.execute("SELECT @@SERVERNAME")
                cursor.fetchone()
                
            self.conn_str = conn_str
            return True
        
        except pyodbc.Error as e:
            print(e)
            return False

    def list_databases(self):


        query = """select [name] 
        from sys.databases
        where name not in ('master','tempdb','model','msdb')
        order by [name]"""

        try:
            with pyodbc.connect(self.conn_str) as conn:
                cursor = conn.cursor()
                cursor.execute(query)
                results = cursor.fetchall()
                results = [result[0] for result in results]
                print(results)
            return results 
        
        except Exception as e:
            print(str(e))
    
    

    def execute_query(self, database, query):

        # Estabelecer a conexão com o banco
        conn_str = self.conn_str.replace(self.database, database)
        conn = pyodbc.connect(conn_str)
        print(conn_str)
        # Executar a consulta para obter o nome do banco
        cursor = conn.cursor()
        cursor.execute("SELECT DB_NAME()")

        # Obter o nome do banco
        db_name = cursor.fetchone()[0]
        
        try:
            # Executar a consulta principal
            cursor.execute(query)
            cursor.commit()

            # Obter o resultado da consulta
            result = 'Consulta realizada'
           
        except ProgrammingError as e:
            result = str(e)
            
        # Fechar a conexão com o banco
        conn.close()

        # Retornar o nome do banco e o resultado da consulta
        return db_name, result

    
    def execute_query_in_databases(self, databases, query):
        
        for database in databases:
            db_name, result = self.execute_query(database, query)
            print('Database:', db_name)
            print('Result:', result)
            time.sleep(3)

banco = Conexao('Heitor')
banco.test_azure_connection()
banco.list_databases()
query = "CREATE TABLE teste (codigo int)"
db_name, result = banco.execute_query('Mart', query)
print('Database:', db_name)
print('Result:', result)
print('n\n\n\n')


selected_databases = ['teste1', 'teste2', 'teste3', 'teste4']
query = "CREATE TABLE teste (codigo int)"
banco.execute_query_in_databases(selected_databases, query)

['AdventureWorks2019', 'Mart', 'teste1', 'teste2', 'teste3', 'teste4']
DRIVER={SQL Server Native Client 11.0};SERVER=Heitor;DATABASE=Mart;Trusted_Connection=yes;
Database: Mart
Result: ('42S01', "[42S01] [Microsoft][SQL Server Native Client 11.0][SQL Server]Já existe um objeto com nome 'teste' no banco de dados. (2714) (SQLExecDirectW)")
n



DRIVER={SQL Server Native Client 11.0};SERVER=Heitor;DATABASE=teste1;Trusted_Connection=yes;
Database: teste1
Result: ('42S01', "[42S01] [Microsoft][SQL Server Native Client 11.0][SQL Server]Já existe um objeto com nome 'teste' no banco de dados. (2714) (SQLExecDirectW)")
DRIVER={SQL Server Native Client 11.0};SERVER=Heitor;DATABASE=teste2;Trusted_Connection=yes;
Database: teste2
Result: ('42S01', "[42S01] [Microsoft][SQL Server Native Client 11.0][SQL Server]Já existe um objeto com nome 'teste' no banco de dados. (2714) (SQLExecDirectW)")
DRIVER={SQL Server Native Client 11.0};SERVER=Heitor;DATABASE=teste3;Trusted_Connection=yes;
Database: teste3

'DRIVER={SQL Server Native Client 11.0};SERVER=Heitor;DATABASE=master;Trusted_Connection=yes;'